# Imports

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

pd.set_option('max_colwidth',200)
pd.set_option("display.max_rows",101)
pd.set_option("display.max_columns",101)

In [4]:
PATH = "/data/msnow/kaggle/house_prices/"
!ls {PATH}

test.csv  train.csv


# Data Exploration

## Explore the dataset before loading all the rows

Determine the number of lines in the csv file

In [5]:
tst_filename = f'{PATH}train.csv'
sum(1 for line in open(tst_filename))

1461

In [7]:
train_raw = pd.read_csv(f'{PATH}train.csv', nrows = 10**2)
trh = train_raw.head().T
trh['dtypes'] = train_raw.dtypes
trh

,0,1,2,3,4,dtypes
Id,1,2,3,4,5,int64
MSSubClass,60,20,60,70,60,int64
MSZoning,RL,RL,RL,RL,RL,object
LotFrontage,65,80,68,60,84,float64
LotArea,8450,9600,11250,9550,14260,int64
Street,Pave,Pave,Pave,Pave,Pave,object
Alley,NaN,NaN,NaN,NaN,NaN,object
LotShape,Reg,Reg,IR1,IR1,IR1,object
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl,object
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub,object


In [9]:
any(x%1 for x in train_raw.LotFrontage)

True

In [21]:
train_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,100.0,50.500000,29.011492,1.0,25.75,50.5,75.25,100.0
MSSubClass,100.0,52.000000,42.545282,20.0,20.00,45.0,60.00,190.0
LotFrontage,86.0,71.406977,21.093840,21.0,60.00,70.0,84.00,122.0
LotArea,100.0,10115.870000,5130.541763,1596.0,7643.25,9595.5,11243.25,50271.0
OverallQual,100.0,5.980000,1.530696,3.0,5.00,6.0,7.00,10.0
OverallCond,100.0,5.500000,0.979590,2.0,5.00,5.0,6.00,8.0
YearBuilt,100.0,1971.410000,29.373284,1910.0,1954.75,1970.0,2000.25,2009.0
YearRemodAdd,100.0,1983.920000,20.967353,1950.0,1964.75,1994.0,2003.25,2009.0
MasVnrArea,100.0,121.640000,213.553982,0.0,0.00,0.0,188.50,1115.0
BsmtFinSF1,100.0,454.560000,462.034922,0.0,0.00,416.0,737.50,1880.0


## Load the full dataset

In [10]:
train_raw = pd.read_csv(f'{PATH}train.csv')
train_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [11]:
data_dict = {"SalePrice": "the property's sale price in dollars. This is the target variable that you're trying to predict.","MSSubClass": "The building class","MSZoning": "The general zoning classification","LotFrontage": "Linear feet of street connected to property","LotArea": "Lot size in square feet","Street": "Type of road access","Alley": "Type of alley access","LotShape": "General shape of property","LandContour": "Flatness of the property","Utilities": "Type of utilities available","LotConfig": "Lot configuration","LandSlope": "Slope of property","Neighborhood": "Physical locations within Ames city limits","Condition1": "Proximity to main road or railroad","Condition2": "Proximity to main road or railroad (if a second is present)","BldgType": "Type of dwelling","HouseStyle": "Style of dwelling","OverallQual": "Overall material and finish quality","OverallCond": "Overall condition rating","YearBuilt": "Original construction date","YearRemodAdd": "Remodel date","RoofStyle": "Type of roof","RoofMatl": "Roof material","Exterior1st": "Exterior covering on house","Exterior2nd": "Exterior covering on house (if more than one material)","MasVnrType": "Masonry veneer type","MasVnrArea": "Masonry veneer area in square feet","ExterQual": "Exterior material quality","ExterCond": "Present condition of the material on the exterior","Foundation": "Type of foundation","BsmtQual": "Height of the basement","BsmtCond": "General condition of the basement","BsmtExposure": "Walkout or garden level basement walls","BsmtFinType1": "Quality of basement finished area","BsmtFinSF1": "Type 1 finished square feet","BsmtFinType2": "Quality of second finished area (if present)","BsmtFinSF2": "Type 2 finished square feet","BsmtUnfSF": "Unfinished square feet of basement area","TotalBsmtSF": "Total square feet of basement area","Heating": "Type of heating","HeatingQC": "Heating quality and condition","CentralAir": "Central air conditioning","Electrical": "Electrical system","1stFlrSF": "First Floor square feet","2ndFlrSF": "Second floor square feet","LowQualFinSF": "Low quality finished square feet (all floors)","GrLivArea": "Above grade (ground) living area square feet","BsmtFullBath": "Basement full bathrooms","BsmtHalfBath": "Basement half bathrooms","FullBath": "Full bathrooms above grade","HalfBath": "Half baths above grade","Bedroom": "Number of bedrooms above basement level","Kitchen": "Number of kitchens","KitchenQual": "Kitchen quality","TotRmsAbvGrd": "Total rooms above grade (does not include bathrooms)","Functional": "Home functionality rating","Fireplaces": "Number of fireplaces","FireplaceQu": "Fireplace quality","GarageType": "Garage location","GarageYrBlt": "Year garage was built","GarageFinish": "Interior finish of the garage","GarageCars": "Size of garage in car capacity","GarageArea": "Size of garage in square feet","GarageQual": "Garage quality","GarageCond": "Garage condition","PavedDrive": "Paved driveway","WoodDeckSF": "Wood deck area in square feet","OpenPorchSF": "Open porch area in square feet","EnclosedPorch": "Enclosed porch area in square feet","3SsnPorch": "Three season porch area in square feet","ScreenPorch": "Screen porch area in square feet","PoolArea": "Pool area in square feet","PoolQC": "Pool quality","Fence": "Fence quality","MiscFeature": "Miscellaneous feature not covered in other categories","MiscVal": "Dollar Value of miscellaneous feature","MoSold": "Month Sold","YrSold": "Year Sold","SaleType": "Type of sale","SaleCondition": "Condition of sale"}

# Initial Processing

Convert the outcome of interest to the metric asked for, i.e., RMSLE of SalePrice

In [12]:
train_raw.SalePrice = np.log(train_raw.SalePrice)

Look at columns with missing data

In [18]:
train_raw.isnull().sum().sort_index()/len(train_raw)

1stFlrSF         0.000000
2ndFlrSF         0.000000
3SsnPorch        0.000000
Alley            0.937671
BedroomAbvGr     0.000000
BldgType         0.000000
BsmtCond         0.025342
BsmtExposure     0.026027
BsmtFinSF1       0.000000
BsmtFinSF2       0.000000
BsmtFinType1     0.025342
BsmtFinType2     0.026027
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
BsmtQual         0.025342
BsmtUnfSF        0.000000
CentralAir       0.000000
Condition1       0.000000
Condition2       0.000000
Electrical       0.000685
EnclosedPorch    0.000000
ExterCond        0.000000
ExterQual        0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
Fence            0.807534
FireplaceQu      0.472603
Fireplaces       0.000000
Foundation       0.000000
FullBath         0.000000
Functional       0.000000
GarageArea       0.000000
GarageCars       0.000000
GarageCond       0.055479
GarageFinish     0.055479
GarageQual       0.055479
GarageType       0.055479
GarageYrBlt      0.055479
GrLivArea   

Convert strings to pandas categories

In [13]:
train_cats(train_raw)

Replace categories with numeric codes, handle missing continuous values and split off dependent variable

In [15]:
df_train, y, nas = proc_df(train_raw, 'SalePrice')

In [17]:
nas

{'GarageYrBlt': 1980.0, 'LotFrontage': 69.0, 'MasVnrArea': 0.0}